In [82]:
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.initializers import Constant
from keras.layers import TextVectorization
import numpy as np
import pickle
import json

In [83]:
X = np.load('data/X.npy', allow_pickle=True)
y = np.load('data/y.npy')
embedding_matrix = np.load('data/embedding_matrix.npy')
num_tokens = np.load('data/num_tokens.npy').item()
embedding_dim = np.load('data/embedding_dim.npy').item()
max_tokens = np.load('data/max_tokens.npy').item()
max_len = np.load('data/max_len.npy').item()

In [84]:
vectorizer = TextVectorization(max_tokens=max_tokens,
                               standardize=None,
                               ngrams=(1, 2),
                               output_sequence_length=max_len)
vectorizer.adapt(X)

2022-02-02 15:39:40.655426: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [85]:
y = y.reshape((-1,1))

In [86]:
print(X.shape)
print(y.shape)

(7613,)
(7613, 1)


In [87]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [88]:
np.save('data/x_train.npy', x_train)
np.save('data/y_train.npy', y_train)
np.save('data/x_test.npy', x_test)
np.save('data/y_test.npy', y_test)

In [89]:
sequences_input = Input(shape=(1), dtype='string')

vectorizer_sequence = vectorizer(sequences_input)

embedding_layer = Embedding(num_tokens,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            trainable=False)
embedded_sequences = embedding_layer(vectorizer_sequence)

x = Dense(130, activation="relu")(embedded_sequences)
x = Dropout(0.2)(x)
x = Dense(60, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(5, activation="relu")(x)
x = Dropout(0.2)(x)

preds = Dense(units=1,
              kernel_initializer='uniform',
              activation='sigmoid')(x)
model = Model(sequences_input, preds)

model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_9 (TextV  (None, 68)               0         
 ectorization)                                                   
                                                                 
 embedding_9 (Embedding)     (None, 68, 200)           10000400  
                                                                 
 dense_41 (Dense)            (None, 68, 130)           26130     
                                                                 
 dropout_32 (Dropout)        (None, 68, 130)           0         
                                                                 
 dense_42 (Dense)            (None, 68, 60)            7860      
                                                           

In [90]:
n_samples = x_train.shape[0]
model_n = 4

history = model.fit(x_train,
                    y_train,
                    validation_split=0.2,
                    batch_size=int(n_samples*0.1),
                    epochs=150)

model.save(f'models/model{model_n}.tf')

with open(f'models/history{model_n}.json', 'w') as f:
    json.dump(history.history, f)

Epoch 1/150


2022-02-02 15:39:42.149928: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 1s 93ms/step - loss: 0.6927 - accuracy: 0.5597 - val_loss: 0.6918 - val_accuracy: 0.5750


2022-02-02 15:39:43.030392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 2/150
9/9 [==============================] - 1s 58ms/step - loss: 0.6915 - accuracy: 0.5609 - val_loss: 0.6896 - val_accuracy: 0.5750
Epoch 3/150
9/9 [==============================] - 1s 57ms/step - loss: 0.6894 - accuracy: 0.5609 - val_loss: 0.6861 - val_accuracy: 0.5750
Epoch 4/150
9/9 [==============================] - 1s 56ms/step - loss: 0.6875 - accuracy: 0.5609 - val_loss: 0.6838 - val_accuracy: 0.5750
Epoch 5/150
9/9 [==============================] - 1s 59ms/step - loss: 0.6864 - accuracy: 0.5609 - val_loss: 0.6824 - val_accuracy: 0.5750
Epoch 6/150
9/9 [==============================] - 1s 57ms/step - loss: 0.6856 - accuracy: 0.5612 - val_loss: 0.6827 - val_accuracy: 0.5754
Epoch 7/150
9/9 [==============================] - 1s 59ms/step - loss: 0.6851 - accuracy: 0.5665 - val_loss: 0.6820 - val_accuracy: 0.5855
Epoch 8/150
9/9 [==============================] - 1s 56ms/step - loss: 0.6841 - accuracy: 0.5713 - val_loss: 0.6801 - val_accuracy: 0.5877
Epoch 9/150
9/9 [===

2022-02-02 15:41:04.169316: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: models/model4.tf/assets
